# CS2545 - Term Mini-Project

### CS2545 - Data Science ###
### Winter, 2025 ###
### UNB, Fredericton ###

Refer to the read me for more details on the project.
</br>
</br>
</br>

### Import Libraries & Save File paths for data

In [14]:
# Importing all the goodies
import pandas as pd
import numpy as np


# ploting imports
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import openpyxl as pxl


# All Data to import using dataframes
population_Data_Path = "../data/population.csv"
Electricity_Generation_Data_Path = "../data/Electricity_Generation.xlsx"
Electricity_Interchange_Data_Path = "../data/Electricity_Interchange.xlsx"
Electricity_Capacity_Data_Path = "../data/Electricity_Capacity.xlsx"
End_Use_Demand_Data_Path = "../data/End_Use_Demand.xlsx"
End_Use_Prices_Data_Path = "../data/End_Use_Prices.xlsx"
energy_trade_canada_path = "../data/energy_trade_canada.xls"
Canadian_Climate_Normals_CANADA_Data_Path = "../data/1991-2020_Canadian_Climate_Normals_CANADA_Data.csv"

### Power Generation Data Set Cleaning 

In [15]:
generation_df = pd.read_excel(Electricity_Generation_Data_Path)

def clean_generation(df, start_row, province_name, num_rows = 8):

    header_row = start_row + 1
    data_rows = start_row + 2

    headers = df.iloc[header_row]
    block = df.iloc[data_rows : data_rows + num_rows].copy()
    block.columns = headers

    block = block.dropna(subset=[block.columns[0]])

    block = block.rename(columns={block.columns[0]: "Energy_Type"})
    block = block.melt(id_vars="Energy_Type", var_name="Year", value_name="GWh")

    block["Province"] = province_name
    block["Year"] = pd.to_numeric(block["Year"], errors="coerce")
    block["GWh"] = pd.to_numeric(block["GWh"], errors="coerce")

    block = block.dropna(subset=["Year", "GWh"])
    block["Year"] = block["Year"].astype(int)

    return block

provinces_generation = {
    "Canada": 5,
    "Newfoundland and Labrador": 16,
    "Prince Edward Island": 27,
    "Nova Scotia": 38,
    "New Brunswick": 49,
    "Quebec": 60,
    "Ontario": 71,
    "Manitoba": 82,
    "Alberta": 93,
    "British Columbia": 104,
    "Saskatchewan": 115,
    "Yukon": 126,
    "Northwest Territories": 137,
    "Nunavut": 148,
}

all_province_data = []

for province, start_row in provinces_generation.items():
    cleaned = clean_generation(generation_df, start_row, province)
    all_province_data.append(cleaned)


gen_clean = pd.concat(all_province_data, ignore_index=True)
quebec_df = gen_clean[gen_clean["Province"] == "Canada"]





### Power Capacity Data Set Cleaning

In [16]:
# Im using the same function from generation data
capacity_df = pd.read_excel(Electricity_Capacity_Data_Path)

provinces_cap = {
    "Canada": 5,
    "Newfoundland and Labrador": 16,
    "Prince Edward Island": 27,
    "Nova Scotia": 38,
    "New Brunswick": 49,
    "Quebec": 60,
    "Ontario": 71,
    "Manitoba": 82,
    "Alberta": 93,
    "British Columbia": 104,
    "Saskatchewan": 115,
    "Yukon": 126,
    "Northwest Territories": 137,
    "Nunavut": 148,
}

all_province_cap_data = []
for province, start_row in provinces_cap.items():
    cap_cleaned = clean_generation(capacity_df, start_row, province)
    all_province_cap_data.append(cap_cleaned)

cap_clean = pd.concat(all_province_cap_data, ignore_index=True)
# cap_clean

### Power Interchange Data Set Cleaning

In [17]:
# Im gonna reuse the same function as well
interchange_df = pd.read_excel(Electricity_Interchange_Data_Path)
provinces_interchange = {
    "Canada": 5,
    "Newfoundland and Labrador": 14,
    "Prince Edward Island": 23,
    "Nova Scotia": 32,
    "New Brunswick": 41,
    "Quebec": 50,
    "Ontario": 59,
    "Manitoba": 68,
    "Alberta": 77,
    "British Columbia": 86,
    "Saskatchewan": 95,
}

all_province_interchange_data = []
for province, start_row in provinces_interchange.items():
    clean_interchange_block = clean_generation(interchange_df, start_row, province)
    all_province_interchange_data.append(clean_interchange_block)

interchange_clean = pd.concat(all_province_interchange_data, ignore_index=True)
# display(interchange_clean)




### End use demand data set cleaning

In [18]:
demand_df = pd.read_excel(End_Use_Demand_Data_Path)

def clean_end_use_demand(df, sector_blocks, num_rows=7):
    all_sectors = []

    for sector, start_row in sector_blocks.items():
        header_row = start_row + 1
        data_start = start_row + 2

        headers = df.iloc[header_row]
        block = df.iloc[data_start : data_start + num_rows].copy()
        block.columns = headers

        block = block.dropna(subset=[block.columns[0]])
        block = block.rename(columns={block.columns[0]: "Energy_Type"})

        block = block.melt(id_vars="Energy_Type", var_name="Year", value_name="PJ")
        block["Sector"] = sector
        block["Year"] = pd.to_numeric(block["Year"], errors="coerce")
        block["PJ"] = pd.to_numeric(block["PJ"], errors="coerce")

        block = block.dropna(subset=["Year", "PJ"])
        block["Year"] = block["Year"].astype(int)

        all_sectors.append(block)

    return pd.concat(all_sectors, ignore_index=True)


sector_blocks = {
    "Total End-Use": 5,
    "Residential": 14,
    "Commercial": 23,
    "Industrial": 32,
    "Transportation": 41
}
demand_clean = clean_end_use_demand(demand_df, sector_blocks)
# display(demand_clean)



### End use Price data set cleaning

In [19]:
prices_df = pd.read_excel(End_Use_Prices_Data_Path)

prices_provinces = {
    "Newfoundland and Labrador": 5,
    "Prince Edward Island": 11,
    "Nova Scotia": 17,
    "New Brunswick": 23,
    "Quebec": 29,
    "Ontario": 35,
    "Manitoba": 41,
    "Alberta": 47,
    "British Columbia": 53,
    "Saskatchewan": 59,
    "Yukon": 65,
    "Northwest Territories": 71,
    "Nunavut": 77,
}


all_province_price_data = []

for province, start_row in prices_provinces.items():
    clean_block = clean_generation(prices_df, start_row, province, num_rows=3)
    all_province_price_data.append(clean_block)

prices_clean = pd.concat(all_province_price_data, ignore_index=True)
# display(prices_clean)

### Climate Data Cleaning Function

In [20]:
climate_df = pd.read_csv(Canadian_Climate_Normals_CANADA_Data_Path, sep=",")

def clean_climate_data(df):

    df.columns = df.columns.str.strip()
    df_yearly = df.copy()
    df_yearly = df_yearly.rename(columns={
        "LOCATION_NAME": "Location",
        "PROVINCE_OR_TERRITORY": "Province",
        "NORMALS_ELEMENT": "Element",
        "Year": "Value"
    })

    df_yearly["Value"] = pd.to_numeric(df_yearly["Value"], errors="coerce")
    df_yearly = df_yearly.dropna(subset=["Value"])

    df_yearly = df_yearly[~df_yearly["Element"].str.contains("Date|StdDev|Extreme", na=False)]

    return df_yearly[["Location", "Province", "PERIOD_OF_RECORD", "ELEMENT_GROUP", "Element", "Value"]]


climate_clean = clean_climate_data(climate_df)




### Population Data Set Cleaning

In [ ]:

def clean_population_database_format(filepath):


    df = pd.read_csv(filepath)

    df = df[["REF_DATE", "GEO", "VALUE"]].rename(columns={
        "REF_DATE": "Date",
        "GEO": "Province",
        "VALUE": "Population"
    })

    df = df.dropna(subset=["Population"])

    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year

    df_yearly = (
        df.groupby(["Province", "Year"], as_index=False)
        .agg({"Population": "mean"})
    )

    return df_yearly

population_yearly_df = clean_population_database_format(population_Data_Path)
population_yearly_df



